In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Create the token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-08-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)


# Create the writer agent
writer = AssistantAgent("writer", model_client=client, system_message="Draft a short paragraph on climate change.")

# Create two editor agents
editor1 = AssistantAgent("editor1", model_client=client, system_message="Edit the paragraph for grammar.")

editor2 = AssistantAgent("editor2", model_client=client, system_message="Edit the paragraph for style.")

# Create the final reviewer agent
final_reviewer = AssistantAgent(
    "final_reviewer",
    model_client=client,
    system_message="Consolidate the grammar and style edits into a final version.",
)

# Build the workflow graph
builder = DiGraphBuilder()
builder.add_node(writer).add_node(editor1).add_node(editor2).add_node(final_reviewer)

# Fan-out from writer to editor1 and editor2
builder.add_edge(writer, editor1)
builder.add_edge(writer, editor2)

# Fan-in both editors into final reviewer
builder.add_edge(editor1, final_reviewer)
builder.add_edge(editor2, final_reviewer)

# Build and validate the graph
graph = builder.build()

# Create the flow
flow = GraphFlow(
    participants=builder.get_participants(),
    graph=graph,
)

# Run the workflow
await Console(flow.run_stream(task="Write a short paragraph about climate change."))


---------- TextMessage (user) ----------
Write a short paragraph about climate change.
---------- TextMessage (writer) ----------
Climate change refers to significant alterations in global temperatures and weather patterns over time, primarily driven by human activities such as burning fossil fuels, deforestation, and industrial processes. These actions release greenhouse gases like carbon dioxide and methane into the atmosphere, leading to an enhanced greenhouse effect that warms the planet. As a result, we are witnessing more frequent and severe weather events, rising sea levels, and disruptions to ecosystems and biodiversity. Urgent action is needed to mitigate its impacts through transitioning to renewable energy sources, promoting sustainable practices, and increasing awareness and adaptation efforts globally.
---------- TextMessage (editor1) ----------
Climate change refers to significant alterations in global temperatures and weather patterns over time, primarily driven by human

TaskResult(messages=[TextMessage(id='320d4a34-167d-46d4-8424-1ed27defb866', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 49, 59, 217863, tzinfo=datetime.timezone.utc), content='Write a short paragraph about climate change.', type='TextMessage'), TextMessage(id='8de7b0a2-5191-4d77-ab0a-3548396882d6', source='writer', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=112), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 50, 11, 7764, tzinfo=datetime.timezone.utc), content='Climate change refers to significant alterations in global temperatures and weather patterns over time, primarily driven by human activities such as burning fossil fuels, deforestation, and industrial processes. These actions release greenhouse gases like carbon dioxide and methane into the atmosphere, leading to an enhanced greenhouse effect that warms the planet. As a result, we are witnessing more frequent and severe weather events, rising sea leve